# Librerías.

In [1]:
import requests,codecs,re,nltk,gensim,string
import pandas as pd
import numpy as np
from lxml import html
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

Importar modulos creados por nosotros

In [11]:
import funciones as fn #funciones es el nombre del script donde hemos creado las funciones

# Escrapeo.

CSV.

In [2]:
path_dataset = '..\\datosTransformados\\scrapping_es\\'
file_name = 'ofertas.csv'

ofertas=pd.read_csv(path_dataset+file_name,sep=';') 
urls=ofertas['url']

Escrapeo.

In [3]:
texto=[]
for url in urls:
    status=requests.get(url).status_code
    if(status==200):
        print('Página disponible.')
        if(url.__contains__('linkedin')):
            pagina=requests.get(url)
            tree=html.fromstring(pagina.content)
            contenido=tree.xpath('//div[@class="show-more-less-html_markup"]/ul/*')
            for elemento in contenido:
                texto.append(str(elemento.text).lower())
        if(url.__contains__('careers.google')):
            pagina=requests.get(url)
            tree=html.fromstring(pagina.content)
            contenido=tree.xpath('//div[@itemprop="qualification"]/ul/*')
            for elemento in contenido:
                texto.append(str(elemento.text).lower())
        if(url.__contains__('lifeatspotify.com')):
            pagina=requests.get(url)
            tree=html.fromstring(pagina.content)
            contenido=tree.xpath('//ul[@class="line-150 size-6 size-7-mobile list_list__mHc5U"]/*')
            for elemento in contenido:
                texto.append(str(elemento.text.lower()))
    else:
        print('Página no disponible.')

Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página no disponible.
Página no disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.
Página disponible.


# Limpieza y tratamiento.

Habilidades inferidas por el equipo.

In [4]:
skills=['r','python','hadoop','spark','machine','learning','ml','sql','statistics','analytics','analyses','analysis','business',
        'datasets','economics','coding','mathematics','visualizations','pandas','visualization','optimization','pandasdata',
        'visualisation','validate','regression','scala','java','qlik','dashboards','crossdisciplinary','analyzing','analytical',
        'analyticscoding','cloudbased','bi','innovations','models','modelling','modeling','model','matplotlib','forecasting',
        'ggplot','improving','improvement','ggplot2','hypotheses'] #Habilidades técnicas.
soft_skills=['team','competence','communicative','collaborate','discipline','efficient','effectively','communicator',
            'communicated','leads','leadership','initiatives','experienced'] #Habilidades personales.

Limpieza y preparación.

In [5]:
stopwords=stopwords.words('english')
def limpiar_string(text):
    text=''.join([word for word in text if word not in string.punctuation]) #Limpiar puntuación.
    text=' '.join([word for word in text.split() if word not in stopwords]) #Limpiar palabras vacías.
    return text
limpio=list(map(limpiar_string,texto))
limpio_suma=''
for line in limpio:
    limpio_suma=limpio_suma+' '+line
limpio_token=word_tokenize(limpio_suma)

Estadísticos.

In [6]:
#HABILIDADES TÉCNICAS.
#Filtrado.
lista_skills=[]
for a in limpio_token:
    if a in skills:
        lista_skills.append(a)
#Preparar DF.
df_skills=pd.DataFrame()
df_skills['habilidad']=lista_skills
df_skills['habilidad']=pd.Categorical(df_skills['habilidad'])
#Unificar variantes de palabras.
df_skills.replace(['analyticscoding','analysis','analyses','analyzing','analytical'],'analytics',inplace=True) #Analytics.
df_skills.replace(['visualizations','visualisation'],'visualization',inplace=True) #Visualization.
df_skills.replace(['models','modelling','modeling'],'models',inplace=True) #Models.
df_skills.replace(['improving','improve','optimization'],'improvement',inplace=True) #Improvement.
df_skills.replace('ggplot2','ggplot',inplace=True) #Ggplot.
df_skills.replace('pandasdata','pandas',inplace=True) #Pandas.
df_skills.replace(['machine','learning'],'ml',inplace=True) #ML.


In [7]:
#Conteo.
df_skill_count=df_skills['habilidad'].value_counts().to_frame().reset_index()
df_skill_count.columns=['habilidad','conteo']
df_skill_count['importancia %']=round((df_skill_count['conteo']/sum(df_skill_count['conteo']))*100,2)
df_skill_count

,habilidad,conteo,importancia %
0,analytics,26,24.76
1,r,6,5.71
2,statistics,6,5.71
3,business,5,4.76
4,datasets,5,4.76
5,python,5,4.76
6,visualization,5,4.76
7,sql,5,4.76
8,economics,4,3.81
9,ml,4,3.81


In [8]:
df_skill_count[df_skill_count['conteo']>1] #Habilidades medianamente importantes (aparecen más de 1 vez).

,habilidad,conteo,importancia %
0,analytics,26,24.76
1,r,6,5.71
2,statistics,6,5.71
3,business,5,4.76
4,datasets,5,4.76
5,python,5,4.76
6,visualization,5,4.76
7,sql,5,4.76
8,economics,4,3.81
9,ml,4,3.81


In [9]:
#HABILIDADES PERSONALES.
lista_soft_skills=[]
for b in limpio_token:
    if b in soft_skills:
        lista_soft_skills.append(b)
df_soft_skills=pd.DataFrame()
df_soft_skills['habilidad']=lista_soft_skills
df_soft_skills['habilidad']=pd.Categorical(df_soft_skills['habilidad'])
#Unificar variantes de palabras.
df_soft_skills.replace(['communicative','communicated','communicator'],'communication',inplace=True)
df_soft_skills.replace('leads','leadership',inplace=True)
df_soft_skills.replace('team','collaborate',inplace=True)

#Conteo.
df_soft_skill_count=df_soft_skills['habilidad'].value_counts().to_frame().reset_index()
df_soft_skill_count.columns=['habilidad','conteo']
df_soft_skill_count['importancia %']=round((df_soft_skill_count['conteo']/sum(df_soft_skill_count['conteo']))*100,2)
df_soft_skill_count

,habilidad,conteo,importancia %
0,collaborate,7,31.82
1,communication,5,22.73
2,competence,3,13.64
3,leadership,2,9.09
4,initiatives,1,4.55
5,experienced,1,4.55
6,efficient,1,4.55
7,effectively,1,4.55
8,discipline,1,4.55


In [10]:
df_soft_skill_count[df_soft_skill_count['conteo']>1] #Habilidades medianamente importantes (aparecen más de 1 vez).

,habilidad,conteo,importancia %
0,collaborate,7,31.82
1,communication,5,22.73
2,competence,3,13.64
3,leadership,2,9.09


Exportamos los datos a archivos .csv

In [16]:
path_dataset2 = '..\\datosTransformados\\scrapping_es\\' # ruta donde se van a guardar los datos .csv
fn.convertir_csv(df_skill_count, 'skill.csv',path_dataset = path_dataset2)
fn.convertir_csv(df_soft_skill_count, 'soft_skill.csv',path_dataset = path_dataset2)